<a href="https://colab.research.google.com/github/O-Kpy/Kaggle/blob/main/Dacon_%EC%8B%AC%EB%A6%AC_%EC%84%B1%ED%96%A5_%EC%98%88%EC%B8%A1_AI_%EA%B2%BD%EC%A7%84%EB%8C%80%ED%9A%8C_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98(%EC%A0%84%EC%B2%98%EB%A6%ACver).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('whitegrid')
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, robust_scale

In [ ]:
print(torch.__version__)

1.8.1+cu101


In [46]:
train = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/심리 성향 예측 AI 경진대회_train.csv')
test = pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/심리 성향 예측 AI 경진대회_test_x.csv')
print(train.shape, test.shape)

(45532, 78) (11383, 77)


In [34]:
data = pd.concat([train, test])

In [49]:
for i in train.select_dtypes('object').columns:
  train = pd.get_dummies(data=train, columns=[i])

for i in test.select_dtypes('object').columns:
  test = pd.get_dummies(data=test, columns=[i])

In [50]:
train['moderate tendencies'] = train['tp06'] + train['tp07'] + train['tp08'] + train['tp09'] + train['tp10']
test['moderate tendencies'] = test['tp06'] + test['tp07'] + test['tp08'] + test['tp09'] + test['tp10']

In [51]:
train['radical tendencies'] = train['tp01'] + train['tp02']	+ train['tp03'] + train['tp04'] + train['tp05']
test['radical tendencies'] = test['tp01'] + test['tp02']	+ test['tp03'] + test['tp04'] + test['tp05']

In [52]:
train_columns = [c for c in train.columns if c.startswith('Q')]
train = train.drop(columns=train_columns)

test_columns = [c for c in test.columns if c.startswith('Q')]
test = test.drop(columns=test_columns)

In [26]:
#@title
labels = ['a', 'b', 'c', 'd', 'e', 'f']
data['QaE_cut'] = pd.qcut(data['QaE'], 6, labels=labels)
data['QbE_cut'] = pd.qcut(data['QbE'], 6, labels=labels)
data['QcE_cut'] = pd.qcut(data['QcE'], 6, labels=labels)
data['QdE_cut'] = pd.qcut(data['QdE'], 6, labels=labels)
data['QeE_cut'] = pd.qcut(data['QeE'], 6, labels=labels)
data['QfE_cut'] = pd.qcut(data['QfE'], 6, labels=labels)
data['QgE_cut'] = pd.qcut(data['QgE'], 6, labels=labels)
data['QhE_cut'] = pd.qcut(data['QhE'], 6, labels=labels)
data['QiE_cut'] = pd.qcut(data['QiE'], 6, labels=labels)
data['QjE_cut'] = pd.qcut(data['QjE'], 6, labels=labels)
data['QkE_cut'] = pd.qcut(data['QkE'], 6, labels=labels)
data['QlE_cut'] = pd.qcut(data['QlE'], 6, labels=labels)
data['QmE_cut'] = pd.qcut(data['QmE'], 6, labels=labels)
data['QnE_cut'] = pd.qcut(data['QnE'], 6, labels=labels)
data['QoE_cut'] = pd.qcut(data['QoE'], 6, labels=labels)
data['QpE_cut'] = pd.qcut(data['QpE'], 6, labels=labels)
data['QqE_cut'] = pd.qcut(data['QqE'], 6, labels=labels)
data['QrE_cut'] = pd.qcut(data['QrE'], 6, labels=labels)
data['QsE_cut'] = pd.qcut(data['QsE'], 6, labels=labels)
data['QtE_cut'] = pd.qcut(data['QtE'], 6, labels=labels)

In [40]:
train = train.drop(index=train[train['familysize'] > 50].index)
test = test.drop(index=test[test['familysize'] > 50].index)

In [41]:
data.shape

(56911, 64)

In [60]:
train_agg = train.drop(columns='voted').groupby('index').agg(['min', 'max', 'sum', 'count', 'median'])
test_agg = test.groupby('index').agg(['min', 'max', 'sum', 'count', 'median'])

train_agg_columns = []
test_agg_columns = []
for i in train_agg.columns.levels[0]:
  for j in train_agg.columns.levels[1]:
    train_agg_columns.append(f'{i}-{j}')

for a in test_agg.columns.levels[0]:
  for b in test_agg.columns.levels[1]:
    test_agg_columns.append(f'{a}-{b}')

train_agg.columns = train_agg_columns
test_agg.columns = test_agg_columns

In [68]:
corr_matrix_train = train_agg.corr()
upper_train = corr_matrix_train.where(np.triu(np.ones(corr_matrix_train.shape), k=1).astype(np.bool))
to_drop_train = [c for c in upper_train.columns if any(abs(upper_train[c])>0.90)]

corr_matrix_test = test_agg.corr()
upper_test = corr_matrix_test.where(np.triu(np.ones(corr_matrix_test.shape), k=1).astype(np.bool))
to_drop_test = [c for c in upper_test.columns if any(abs(upper_test[c])>0.90)]

print(f'train_agg whole columns:{len(train_agg.columns)}, train to drop columns:{len(to_drop_train)} and test_agg whole columns:{len(test_agg.columns)}, test to drop columns:{len(to_drop_test)}')

train_agg whole columns:310, train to drop columns:187 and test_agg whole columns:310, test to drop columns:187


In [69]:
train_final = train.merge(train_agg.drop(columns=to_drop_train), how='left', on='index')
test_final = test.merge(test_agg.drop(columns=to_drop_test), how='left', on='index')

In [77]:
x_train = train_final.drop(columns=['index', 'voted'])
y_train = 2 - train_final['voted']
x_test = test_final.drop(columns=['index'])
y_train = y_train.astype(np.float32)

In [78]:
train_columns = x_train.columns
x_train = robust_scale(x_train)
x_test = robust_scale(x_test)

In [79]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(222, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, 1)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = F.relu(self.fc2(x))
    x = self.dropout(x)
    x = self.fc3(x)
    return x
model=Net()
print(model)

Net(
  (fc1): Linear(in_features=222, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [15]:
from sklearn.model_selection import StratifiedKFold
import random
from datetime import datetime

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from tqdm import tqdm

In [81]:
train_y_t = torch.tensor(y_train, dtype=torch.float32)
train_x_t = torch.tensor(x_train, dtype=torch.float32)
test_x_t = torch.tensor(x_test, dtype=torch.float32)

test_len = len(test_x_t)

In [17]:
train_x_t[:, :20] = (train_x_t[:, :20] - 3.) / 2.
test_x_t[:, :20] = (test_x_t[:, :20] - 3.) / 2
train_x_t[:, 20] = (train_x_t[:, 20] - 5.) / 4.
test_x_t[:, 20] = (test_x_t[:, 20] - 5.) / 4.
train_x_t[:, 21:31] = (train_x_t[:, 21:31] - 3.5) / 3.5
test_x_t[:, 21:31] = (test_x_t[:, 21:31] - 3.5) / 3.5

In [83]:
x_train.shape

(45532, 185)

In [84]:
N_REPEAT = 5
N_SKFOLD = 7
N_EPOCH = 48
BATCH_SIZE = 72
LOADER_PARAM = {
    'batch_size': BATCH_SIZE,
    'num_workers': 4,
    'pin_memory': True
}

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
prediction = np.zeros((test_len, 1), dtype=np.float32)

for repeat in range(N_REPEAT):

    skf, tot = StratifiedKFold(n_splits=N_SKFOLD, random_state=repeat, shuffle=True), 0.
    for skfold, (train_idx, valid_idx) in enumerate(skf.split(x_train, y_train)):
        train_idx, valid_idx = list(train_idx), list(valid_idx)
        train_loader = DataLoader(TensorDataset(train_x_t[train_idx, :], train_y_t[train_idx]),
                                  shuffle=True, drop_last=True, **LOADER_PARAM)
        valid_loader = DataLoader(TensorDataset(train_x_t[valid_idx, :], train_y_t[valid_idx]),
                                  shuffle=False, drop_last=False, **LOADER_PARAM)
        test_loader = DataLoader(TensorDataset(test_x_t, torch.zeros((test_len,), dtype=torch.float32)),
                                 shuffle=False, drop_last=False, **LOADER_PARAM)
        model = nn.Sequential(
            nn.Dropout(0.05),
            nn.Linear(185, 400, bias=False),
            nn.LeakyReLU(0.05, inplace=True),
            nn.Dropout(0.5),
            nn.Linear(400, 222, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(222, 1)
        ).to(DEVICE)
        criterion = torch.nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.20665], device=DEVICE))
        optimizer = optim.AdamW(model.parameters(), lr=5e-3, weight_decay=7.8e-2)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer, T_0=N_EPOCH // 6, eta_min=4e-4)
        prediction_t, loss_t = np.zeros((test_len, 1), dtype=np.float32), 1.

        # for epoch in range(N_EPOCH):
        for epoch in tqdm(range(N_EPOCH), desc='{:02d}/{:02d}'.format(skfold + 1, N_SKFOLD)):
            model.train()
            for idx, (xx, yy) in enumerate(train_loader):
                optimizer.zero_grad() ## 최적화 준비
                xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                pred = model(xx).squeeze() ## predict뽑아내기
                loss = criterion(pred, yy) ## loss 뽑아내기
                loss.backward()
                optimizer.step() ## 최적화로 뒤로간다
                scheduler.step(epoch + idx / len(train_loader))

            with torch.no_grad():
                model.eval()
                running_acc, running_loss, running_count = 0, 0., 0
                for xx, yy in valid_loader:
                    xx, yy = xx.to(DEVICE), yy.to(DEVICE)
                    pred = model(xx).squeeze()
                    loss = criterion(pred, yy)
                    running_loss += loss.item() * len(yy)
                    running_count += len(yy)
                    running_acc += ((torch.sigmoid(pred) > 0.5).float() == yy).sum().item()
                # print('R{:02d} S{:02d} E{:02d} | {:6.4f}, {:5.2f}%'
                #       .format(repeat + 1, skfold + 1, epoch + 1, running_loss / running_count,
                #               running_acc / running_count * 100))

                if running_loss / running_count < loss_t:
                    loss_t = running_loss / running_count
                    for idx, (xx, _) in enumerate(test_loader):
                        xx = xx.to(DEVICE)
                        pred = (2. - torch.sigmoid(model(xx).detach().to('cpu'))).numpy()
                        prediction_t[BATCH_SIZE * idx:min(BATCH_SIZE * (idx + 1), len(prediction)), :] \
                            = pred[:, :].copy()
        prediction[:, :] += prediction_t[:, :].copy() / (N_REPEAT * N_SKFOLD)
        tot += loss_t
    print('R{} -> {:6.4f}'.format(repeat + 1, tot / N_SKFOLD))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
07/07: 100%|██████████| 48/48 [03:29<00:00,  4.37s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
01/07:   0%|          | 0/48 [00:

R1 -> 0.6119


07/07: 100%|██████████| 48/48 [03:49<00:00,  4.78s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R2 -> 0.6125


07/07: 100%|██████████| 48/48 [04:08<00:00,  5.18s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R3 -> 0.6125


07/07: 100%|██████████| 48/48 [04:10<00:00,  5.21s/it]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
01/07:   0%|          | 0/48 [00:00<?, ?it/s]

R4 -> 0.6117


07/07: 100%|██████████| 48/48 [04:25<00:00,  5.53s/it]

R5 -> 0.6122


In [85]:
submit=pd.read_csv('/content/drive/MyDrive/dataset/Dacon/연습/심리 성향 예측 AI 경진대회_sample_submission.csv')
submit.iloc[:,1:] = prediction

In [86]:
submit = submit.set_index('index')
submit

,voted
index,
0,1.640144
1,1.917178
2,1.477831
3,1.180768
4,1.805007
...,...
11378,1.378419
11379,1.825347
11380,1.235604


In [87]:
submit.to_csv('/content/drive/MyDrive/dataset/Dacon/연습/2021-05-19 submit3.csv')

In [ ]:
# 너무 오래걸려서 오늘 다 못돌림 내일 하루종일 돌리기